# Table of Contents

0. Preface
  - Instructions
  - Installer
  - Imports
  - Colour scheme preview
1. 4D 
  - Animated rotating wireframe image (projected)
  - Superposed animation frames (projected)
2. 3D Net
  - Wireframe image (projected)
  - Solid cell-shaded image (projected)
  - STL
3. 2D facet nets
  - Plain
  - Solid colour (1 per facet)

## Instructions

1. Run the initialisation cells (down to **Colour scheme preview**) exactly once, before doing anything else.
2. In the **Colour scheme preview** section, run the first cell to display a list of colour schemes. Change the value of `PREVIEW_COLOR_SCHEME` in the second cell and run it to get a preview of that colour scheme.
3. The first half of the **Parameters** cell is where you can set all the options for the output. They should be more or less self explanatory; more will be added later. Note that strange things can happen with the projection is `PERSPECTIVE_DISTANCE` is set too low.
4. Run the **Randomization** cell each time you want to generate a new viewing angle.
5. In the **Output area**, each type of output has a *preview pane* cell and a *save files* cell. Run the preview pane cell and adjust **Parameters** until you find a view you like, then run the save files cell. Files are saved to `SAVE_DIRECTORY` (which is `./output` by default).
   
   Note that it is not possible to preview animations or STL files. For animations, ffmpeg must be installed (this should happen automatically if you are running the notebook on mybinder.org).

In [ ]:
# Installer
import sys
!{sys.executable} -m pip install ..

# install ffmpeg for animations (assuming x86 linux with ~/.local/bin on path)
#!wget https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz
#!unxz ffmpeg-release-amd64-static.tar.xz
#!tar xf ffmpeg-release-amd64-static.tar
#!ln -s $(pwd)/ffmpeg-6.0-amd64-static/ffmpeg $HOME/.local/ffmpeg

# uncomment the next line and rerun in case of ffmpeg troubles
#!conda install -yq -c conda-forge ffmpeg

# IMPORTS
from tope import Tope
from tope.net import *
from tope.orth import *
from tope.graph import Graph
from tope.plot import *

import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import json, os

# import prepackaged data
with open("../data/polys2.json") as fd: 
    polys = json.load(fd)
    
# and the rest
import gzip
with gzip.open("../data/474polys.json.gz", "rt") as fd:
    polys.update(json.load(fd))

with gzip.open("../data/d30.json.gz", "rt") as fd:
    d30 = json.load(fd)
polys.update({f"d30-{record['ID']}": record["Vertices"] for record in d30})

logger.remove()

In [ ]:
def create_lc(edges, color_map = "Set2", color_range=(0.25,0.75), lw=1):
    segments = []
    colors = []
    cmap = mpl.colormaps.get(color_map)
    for i, edge in enumerate(edges):
        segments.append(edge)
        crange_start = color_range[0]
        crange_step = (color_range[1]-color_range[0]) / len(edges)
        colors.append(cmap(crange_start + i*crange_step))
    return mpl.collections.LineCollection(segments, color=colors, linewidth=lw)

def get_wireframe(P: Tope, rotator, perspective_distance=10):
    rotate  = lambda e: e @ rotator
    project = lambda e: perspective_project(perspective_project(e, perspective_distance), perspective_distance)
    return list(map(project, map(rotate, P.iter_faces_as_arrays(dim=1))))

def generate_rotators(N: int, num_steps: int = 10) -> np.ndarray:
    """
    Generate num_steps evenly spaced rotations of stacked vectors.
    """
    return rotator_nd(np.arange(0, 2*np.pi, 2*np.pi / num_steps), N)

## Polytope ID listing

In [ ]:
print("Available polytopes:")
print("--------------------")
for i, cm in enumerate(polys): 
    print(f"{cm:14}", end="")
    if i%8==7: print()
print()
print()
print("Note: all numbers less than 474000000 and ending in 123456 are available.")
print("Entries whose ID has the prefix 'd30-' have at least 30 facets.")

# Colour scheme preview

In [ ]:
print("Available named colors:")
print("-----------------------")
for i, cm in enumerate(mpl.colors.get_named_colors_mapping()): 
    print(f"{cm:24}", end="")
    if i%5==4: print()

In [ ]:
print("Available color schemes:")
print("------------------------")
for i, cm in enumerate(mpl.colormaps): 
    print(f"{cm:20}", end="")
    if i%5==4: print()

#### Paste one of these strings in between the quotation marks and run this cell (Shift+Enter) to preview a colour scheme!

In [ ]:
PREVIEW_COLOR_SCHEME = "Spectral"
mpl.colormaps.get(PREVIEW_COLOR_SCHEME)

# Parameters

In [ ]:
POLYTOPE     = "375123456"

COLOR_SCHEME      = "Pastel1_r"
COLOR_RANGE_BEGIN = 0.25   # between 0 and 1
COLOR_RANGE_END   = 0.75   # between 0 and 1
BG_COLOR         = "xkcd:poo brown"

PERSPECTIVE_DISTANCE = 10

DPI           = 300
ANIMATION_DPI = 150

TAG            = "bum" # put nonempty string here to add custom text to filenames
SAVE_DIRECTORY = "output"



# don't change ##################
os.makedirs(SAVE_DIRECTORY, exist_ok=True)

DIR_4D = os.path.join(SAVE_DIRECTORY, "4d-wireframe")
DIR_ANIMATION = os.path.join(DIR_4D, "animated")
DIR_SMEARED = os.path.join(DIR_4D, "smeared")
DIR_3D = os.path.join(SAVE_DIRECTORY, "3d-net")
DIR_STL = os.path.join(DIR_3D, "stl")
DIR_NET_PROJECTION = os.path.join(DIR_3D, "projected")
DIR_SHADED_3D_NET = os.path.join(DIR_3D, "shaded")
DIR_2D = os.path.join(SAVE_DIRECTORY, "facet-nets")
P = Tope.from_vertices(polys[POLYTOPE])

Q3 = Q3 if 'Q3' in globals() else None
TAG = TAG or ((id(POLYTOPE)+id(Q3))%65536).to_bytes(2,"big").hex()

## Randomization

In [ ]:
Q4a = random_orth(4)
Q4b = random_orth(4)
Q3 = random_orth(3)
I4 = np.eye(4)

TAG = TAG or ((id(POLYTOPE)+id(Q3))%65536).to_bytes(2,"big").hex()

# Output area

# 4D

In [ ]:
def get_frames(P, before, after, num_steps=10):
    return [create_lc(get_wireframe(P, before @ rotator_nd(theta, 4) @ after)) for theta in np.arange(0, 2*np.pi, 2*np.pi / num_steps)]

In [ ]:
def plot_wireframe(
    wf: list[np.ndarray], # iterable yielding 2x2 arrays
    color_map = "Pastel1_r",
    color_range = (0.25,0.75),
    weight = 1,
    bg_color = "beige",
    border = False,
    **kwargs
):
    
    fig, ax = plt.subplots()
    
    lines = create_lc(
        wf,
        color_map = color_map,
        color_range = color_range,
        lw = weight
    )

    ax.add_collection(lines)
    
    ax.autoscale()
    ax = configure_axes(ax, bg=bg_color, border=border)

    return fig, ax

### Preview 4d wireframe

In [ ]:
fig, _ = plot_wireframe(get_wireframe(P, Q4b), border=True, color_map = "Spectral", color_range=(0.75,1), bg_color=BG_COLOR)
fig.set_size_inches(10,10)

In [ ]:
frames = get_frames(P, Q4a, Q4b, num_steps=100)
bbox = get_tightbbox(*frames) # must compute before adding artists to axes!

fig, ax = plt.subplots()
for frame in frames:
    ax.add_artist(frame)
ax.dataLim = bbox
ax = configure_axes(ax, bg=BG_COLOR)

### Save output (smear and animation)
Make sure to run the previous cell first!

In [ ]:
# save smear
os.makedirs(DIR_SMEARED, exist_ok=True)
fig.savefig(os.path.join(DIR_SMEARED, f"{POLYTOPE}-{TAG}.png"), dpi=DPI) 

# save animation
from matplotlib.animation import ArtistAnimation
os.makedirs(DIR_ANIMATION, exist_ok=True)
animation = ArtistAnimation(fig, [[frame] for frame in frames], interval=1000/60)
animation.save(os.path.join(DIR_ANIMATION, f"{POLYTOPE}-{TAG}.mp4"), dpi=ANIMATION_DPI)

# 3D net

In [ ]:
N = P.net().unfold().in_own_span()

### Preview

In [ ]:
fig, ax = plt.subplots(dpi=DPI)

cell_edges = [np.stack([F.vertices[sorted(e)] for e in F.iter_faces(1)]) for F in N.facets.values()]
cmaps = list(mpl.colormaps)

for n, cell in enumerate(cell_edges):
    edges = perspective_project(cell @ Q3, 10)
    lc = create_lc(edges, color_map = cmaps[n%len(cmaps)])
    ax.add_collection(lc)

#ax = configure_axes(ax, bg=BG_COLOR)
limits = ax.axis("scaled")
ax.set_axis_off()
ax.set_facecolor("black")

fig.set_size_inches(20,20)

### Save output
Make sure to run the previous cell first!

In [ ]:
os.makedirs(DIR_NET_PROJECTION, exist_ok=True)
fig.savefig(os.path.join(DIR_NET_PROJECTION, f"{POLYTOPE}-{TAG}.png"), dpi=DPI)

## STL
Export as STL

In [ ]:
from tope.stl import create_stl
thing = create_stl(*N.facets.values())
assert thing.check()

os.makedirs(DIR_STL, exist_ok=True)
thing.save(os.path.join(DIR_STL, f"{POLYTOPE}-{TAG}.stl"))

## Experimental: shaded 3d net

In [ ]:
l = list(N.facets.values())
colors = [mpl.colormaps[COLOR_SCHEME](k/len(l)) for k in range(len(l)) for _ in l[k].triangulate()]

### Preview

In [ ]:
import mpl_toolkits.mplot3d as mpl3d

ar = mpl3d.art3d.Poly3DCollection(thing.vectors, shade=True, lightsource=mpl.colors.LightSource(), facecolors=colors)
fig = plt.figure(dpi=300)
ax = fig.add_subplot(projection='3d')
ax.add_artist(ar)

ax = configure_axes_3d(ax, thing.vectors, bg=BG_COLOR)

### Save

In [ ]:
os.makedirs(DIR_SHADED_3D_NET, exist_ok=True)
fig.savefig(os.path.join(DIR_SHADED_3D_NET, f"{POLYTOPE}-{TAG}.png"), dpi=DPI)

# 2d nets